In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
connection_string = 'postgresql' + '://' + 'buddyadmin' + ':' + \
                    'fintechbuddy' + '@' + 'postgres-buddy.ctzb27aixti7.ap-southeast-1.rds.amazonaws.com:5432' + '/' + \
                    'buddymldb'

In [4]:
engine = create_engine(connection_string)

In [5]:
data_df = pd.read_sql_query("""select * from user_network_details""", con=engine)

In [8]:
data_df.head()

,uuid,network_uuid,network_id
0,8660542698,9611191661,2
1,7795246117,9538629444,2
2,9740320701,9206430835,2
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1
4,7676360933,7795988665,2


In [11]:
data_df.shape

(197370, 3)

In [12]:
credit_score_df = pd.read_sql_query("""SELECT credit_risk_uuid.* FROM credit_risk_uuid, (
    SELECT credit_risk_uuid.uuid as uid, MAX(credit_risk_uuid.created_at) AS c_at 
    FROM credit_risk_uuid 
    GROUP BY credit_risk_uuid.uuid
) a where credit_risk_uuid.uuid = a.uid and credit_risk_uuid.created_at = a.c_at""", con=engine)

In [13]:
credit_score_df.shape

(48618, 7)

In [14]:
result_df = pd.merge(data_df,credit_score_df,how="left",on="uuid")

In [15]:
result_df.drop(['max_dpd_user','model_credit_score','model_credit_score','created_at','model_proba_default','probability_of_default'], axis=1, inplace=True)

In [16]:
result_df.shape

(197370, 4)

In [17]:
result_df.rename(columns = {'credit_score':'user_credit_score'}, inplace=True)

In [18]:
final_df = pd.merge(result_df, credit_score_df, left_on='network_uuid',right_on='uuid', how="left")

In [19]:
final_df.drop(['max_dpd_user','model_credit_score','model_credit_score','created_at','model_proba_default','probability_of_default'], axis=1, inplace=True)

In [20]:
final_df.drop('uuid_y',axis= 1,inplace=True)

In [21]:
final_df.head()

,uuid_x,network_uuid,network_id,user_credit_score,credit_score
0,8660542698,9611191661,2,600.799810,607.021215
1,7795246117,9538629444,2,NaN,799.394880
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
4,7676360933,7795988665,2,NaN,782.319798


In [22]:
final_df.rename(columns = {'credit_score':'user_credit_score','uuid_x':'user_uuid'}, inplace=True)

In [23]:
final_df.duplicated(subset=['user_uuid','network_uuid','network_id'], keep='first').sum()

42858

In [24]:
final_df.loc[~final_df.duplicated(subset=['user_uuid','network_uuid','network_id'], keep='first'),:]

,user_uuid,network_uuid,network_id,user_credit_score,user_credit_score
0,8660542698,9611191661,2,600.799810,607.021215
1,7795246117,9538629444,2,NaN,799.394880
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
4,7676360933,7795988665,2,NaN,782.319798
5,7411631664,8083299245,2,NaN,NaN
6,e8c82bc0-de19-11e6-aa79-1bde32099ed8,7411001943,2,602.678066,785.345390
7,e3f6d850-1db9-11e7-9206-af497475ad17,2c32ee30-0e4f-11e7-a48f-19477d62de23,1,648.787025,782.627034
8,3592a900-2d5a-11e7-9dfe-85394b1021d9,5a664180-f67b-11e6-9388-d588daabb8b4,4,NaN,799.390812
9,0895fb70-d3e4-11e6-b537-3bb6a1158e1a,8296490815,1,786.936908,735.435110


In [25]:
final_df.drop_duplicates(subset=['user_uuid','network_uuid','network_id'], keep='first', inplace=True)

In [26]:
final_df.shape

(154512, 5)

In [27]:
final_df.to_csv('filtered_network_data.csv')

In [28]:
final_df.rename(columns = {'user_credit_score':'user_credit_score','user_credit_score':'network_credit_score'}, inplace=True)

In [29]:
final_df.head()

,user_uuid,network_uuid,network_id,network_credit_score,network_credit_score
0,8660542698,9611191661,2,600.799810,607.021215
1,7795246117,9538629444,2,NaN,799.394880
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
4,7676360933,7795988665,2,NaN,782.319798


In [30]:
final_df.columns

Index([           u'user_uuid',         u'network_uuid',
                 u'network_id', u'network_credit_score',
       u'network_credit_score'],
      dtype='object')

In [31]:
final_df.columns.values[3] = 'user_credit_score'

In [32]:
final_df.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
0,8660542698,9611191661,2,600.799810,607.021215
1,7795246117,9538629444,2,NaN,799.394880
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
4,7676360933,7795988665,2,NaN,782.319798


In [33]:
final_df['user_credit_score'].isnull().sum()

55367

In [34]:
final_df.loc[~final_df['user_credit_score'].isnull(),:]

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
0,8660542698,9611191661,2,600.799810,607.021215
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
6,e8c82bc0-de19-11e6-aa79-1bde32099ed8,7411001943,2,602.678066,785.345390
7,e3f6d850-1db9-11e7-9206-af497475ad17,2c32ee30-0e4f-11e7-a48f-19477d62de23,1,648.787025,782.627034
9,0895fb70-d3e4-11e6-b537-3bb6a1158e1a,8296490815,1,786.936908,735.435110
10,c02c8bb0-1d30-11e7-a1f9-1f21e945de24,2dfefbd0-e011-11e6-89e5-bf4923c964eb,1,799.988640,799.999622
11,3a76b5a0-ea8f-11e6-9cfe-d10c509d1d15,7338527272,1,722.355963,799.999622
12,77b69aa0-e9bf-11e6-8e75-87909e99c2b7,0bab7ea0-ee58-11e6-ac35-7d60aa46fc46,1,798.450852,799.996198
13,05b87910-0def-11e7-aba4-39bc64788177,7411532624,1,799.999622,607.141900


In [35]:
# final_df.loc[~final_df['network_credit_score'].isnull(),:].shape
final_df = final_df.loc[~final_df['user_credit_score'].isnull(),:]

In [36]:
final_df.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
0,8660542698,9611191661,2,600.799810,607.021215
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
6,e8c82bc0-de19-11e6-aa79-1bde32099ed8,7411001943,2,602.678066,785.345390
7,e3f6d850-1db9-11e7-9206-af497475ad17,2c32ee30-0e4f-11e7-a48f-19477d62de23,1,648.787025,782.627034


In [37]:
final_df.network_id.value_counts()

5    26457
3    24687
1    23620
2    18455
4     5926
Name: network_id, dtype: int64

In [38]:
final_df_referral = final_df[final_df.network_id==3]

In [39]:
final_df_referral.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
105,8553383519,9738930151,3,602.180711,NaN
115,f7633360-f9b1-11e6-a9c5-4f0b16583440,5d745180-f9c0-11e6-8558-8f833f3076cd,3,664.116614,603.549948
116,f7633360-f9b1-11e6-a9c5-4f0b16583440,31622790-fdab-11e6-89b6-5ff7811f812a,3,664.116614,605.259469
199,8147807376,7204609571,3,799.995350,NaN
200,8105682408,7760810397,3,642.989050,785.345390


In [40]:
final_df_referral.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
105,8553383519,9738930151,3,602.180711,NaN
115,f7633360-f9b1-11e6-a9c5-4f0b16583440,5d745180-f9c0-11e6-8558-8f833f3076cd,3,664.116614,603.549948
116,f7633360-f9b1-11e6-a9c5-4f0b16583440,31622790-fdab-11e6-89b6-5ff7811f812a,3,664.116614,605.259469
199,8147807376,7204609571,3,799.995350,NaN
200,8105682408,7760810397,3,642.989050,785.345390


In [41]:
final_df_referral.shape

(24687, 5)

In [42]:
df_8152002106 = final_df_referral[final_df_referral.user_uuid=='8152002106']
# final_df_referral.user_uuid.value_counts()

In [43]:
df_8152002106.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
3848,8152002106,7353823625,3,799.980158,799.999622
3850,8152002106,8762151300,3,799.980158,762.046832
3851,8152002106,8197236207,3,799.980158,695.667802
4063,8152002106,8884801621,3,799.980158,601.396469
12153,8152002106,3b458d00-fe88-11e6-89b6-5ff7811f812a,3,799.980158,798.821374


In [44]:
final_df_referral.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
105,8553383519,9738930151,3,602.180711,NaN
115,f7633360-f9b1-11e6-a9c5-4f0b16583440,5d745180-f9c0-11e6-8558-8f833f3076cd,3,664.116614,603.549948
116,f7633360-f9b1-11e6-a9c5-4f0b16583440,31622790-fdab-11e6-89b6-5ff7811f812a,3,664.116614,605.259469
199,8147807376,7204609571,3,799.995350,NaN
200,8105682408,7760810397,3,642.989050,785.345390


In [45]:
df_remove_nan = final_df.loc[~final_df['network_credit_score'].isnull(),:]

In [46]:
df_remove_nan.head()

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
0,8660542698,9611191661,2,600.799810,607.021215
2,9740320701,9206430835,2,798.452916,783.129331
3,644fb3b0-e3cb-11e6-b685-7942eecceaad,44ae7fd0-d306-11e6-b551-2f5c6833db7e,1,600.000000,797.879410
6,e8c82bc0-de19-11e6-aa79-1bde32099ed8,7411001943,2,602.678066,785.345390
7,e3f6d850-1db9-11e7-9206-af497475ad17,2c32ee30-0e4f-11e7-a48f-19477d62de23,1,648.787025,782.627034


In [47]:
df_remove_nan = df_remove_nan[df_remove_nan.network_id==3]

In [48]:
df_remove_nan

,user_uuid,network_uuid,network_id,user_credit_score,network_credit_score
115,f7633360-f9b1-11e6-a9c5-4f0b16583440,5d745180-f9c0-11e6-8558-8f833f3076cd,3,664.116614,603.549948
116,f7633360-f9b1-11e6-a9c5-4f0b16583440,31622790-fdab-11e6-89b6-5ff7811f812a,3,664.116614,605.259469
200,8105682408,7760810397,3,642.989050,785.345390
205,8105682408,7829833982,3,642.989050,784.107971
206,8105682408,9845075927,3,642.989050,651.418879
276,8553229080,5504b750-e1ed-11e6-b685-7942eecceaad,3,608.350119,799.981096
279,8553229080,a6a65210-e1ef-11e6-b685-7942eecceaad,3,608.350119,799.944763
439,8553229080,9313bcf0-e2ae-11e6-b685-7942eecceaad,3,608.350119,799.944763
440,8553229080,428eeac0-e2d6-11e6-b685-7942eecceaad,3,608.350119,780.732688
590,9901908531,7996909083,3,603.578490,605.967513


In [49]:
X = df_remove_nan[['network_credit_score','network_id']]
X.head()

,network_credit_score,network_id
115,603.549948,3
116,605.259469,3
200,785.345390,3
205,784.107971,3
206,651.418879,3


In [50]:
y = df_remove_nan['user_credit_score']

In [58]:
df_remove_nan.to_csv('referral_network.csv')
# from sklearn.linear_model import LinearRegression

In [86]:
counts = df_remove_nan.user_uuid.value_counts()
# df_remove_nan[df_remove_nan.user_uuid.isin(df_remove_nan.user_uuid.value_counts() > 1)]
# counts.head()
counts[counts > 1]
# linreg = LinearRegression()

8880007747                              73
8105734372                              71
34c85c00-e2db-11e6-b685-7942eecceaad    67
9742704753                              59
8880580320                              58
9642490000                              58
9620004029                              57
9738012743                              56
9496334860                              53
8904877907                              52
d5e7a690-e3d0-11e6-b685-7942eecceaad    51
8792636761                              49
9035558959                              43
8152002106                              43
4a60de30-26c3-11e7-9d70-a9f74712d401    42
3187eb00-e470-11e6-93fa-ddfa474de486    42
833661a0-e13e-11e6-b3a5-01d081544184    42
2e287790-eaab-11e6-9cfe-d10c509d1d15    42
9886677046                              40
8892980779                              40
9620195830                              40
7899081088                              38
8986120992                              38
9880971320 

In [53]:
# X.shape


(13226, 2)

In [54]:
# import numpy as np

,network_credit_score,network_id
115,603.549948,3
116,605.259469,3
200,785.345390,3
205,784.107971,3
206,651.418879,3
276,799.981096,3
279,799.944763,3
439,799.944763,3
440,780.732688,3
590,605.967513,3


In [56]:
# X = np.reshape(X.shape[0],1)
# linreg.fit(X,y)

/Library/Python/2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [57]:
# print('Coefficients: \n', linreg.coef_)

('Coefficients: \n', array([ 0.18386862,  0.        ]))
